In [9]:
# --- Бібліотеки для даної роботи ---
try:
    import numpy, pandas, scipy, jinja2
    print("Бібліотеки вже встановлені. Пропускаємо інсталяцію.")
except ImportError:
    print("Встановлюємо бібліотеки...")
    %pip install numpy pandas scipy jinja2

Бібліотеки вже встановлені. Пропускаємо інсталяцію.


## Задача 1. Аналіз динаміки користувацької активності

**Умова:**

Ви працюєте аналітиком в EdTech-компанії та досліджуєте активність користувачів онлайн-платформи протягом дня.

На основі зібраних даних було побудовано модель, яка описує кількість активних сесій:

$$f(t) = 1000 \cdot t \cdot e^{-0.2t}$$

де $t$ — час у годинах від початку робочого дня (8:00), а $f(t)$ — кількість одночасно активних сесій.

Керівництво хоче зрозуміти динаміку навантаження на сервери, щоб оптимально розподілити ресурси.

Для цього потрібно проаналізувати, як швидко змінюється активність у різні моменти дня.

**Використовуючи методи диференціювання, напишіть програму та виконайте наступні кроки:**

1.  **Знайдіть похідну аналітично.** Продиференціюйте функцію $f(t) = 1000 \cdot t \cdot e^{-0.2t}$, використовуючи правило диференціювання добутку: $(u \cdot v)' = u' \cdot v + u \cdot v'$, де $u(t) = 1000t$ та $v(t) = e^{-0.2t}$. Запишіть отриману формулу для $f'(t)$.
2.  **Визначте момент пікового навантаження.** Знайдіть момент часу $t^*$, коли кількість активних сесій досягає максимуму. Для цього розв'яжіть рівняння $f'(t) = 0$. Переведіть результат у формат години:хвилини (наприклад, $t^* = 3.5$ означає 11:30).
3.  **Обчисліть швидкість зміни чисельно.** Використовуючи функцію `approx_fprime` з бібліотеки `scipy.optimize`, обчисліть значення похідної (швидкість зміни кількості сесій) у три моменти:
    * о 10:00 ($t = 2$)
    * о 14:00 ($t = 6$)
    * о 18:00 ($t = 10$)
4.  **Порівняйте чисельні та аналітичні результати.** Обчисліть значення похідної за аналітичною формулою з пункту 1 для тих самих моментів часу ($t = 2, 6, 10$). Порівняйте із чисельними значеннями з пункту 3.
5.  **Інтерпретуйте результати для бізнесу.** На основі отриманих даних дайте відповіді:
    * Що означає додатне значення похідної о 10:00 для IT-відділу?
    * Що означає від'ємне значення похідної о 18:00?
    * О котрій годині варто мати максимальну кількість серверів у роботі?

In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import approx_fprime
from IPython.display import display

def f_task(t):
    return 1000 * t * np.exp(-0.2 * t)

def df_analytical(t):
    return 1000 * np.exp(-0.2 * t) * (1 - 0.2 * t)

t_peak = 5.0
start_hour = 8
peak_hour = int(start_hour + t_peak)
peak_minute = int((t_peak % 1) * 60)
peak_time_str = f"{peak_hour:02d}:{peak_minute:02d}"

check_points = [2, 6, 10] # t=2 (10:00), t=6 (14:00), t=10 (18:00)
epsilon = 1e-6
results = []

for t_val in check_points:
    numeric_deriv = approx_fprime([t_val], f_task, epsilon)[0]
    exact_deriv = df_analytical(t_val)
    clock_time = f"{start_hour + t_val}:00"
    
    results.append({
        "t (год)": t_val,
        "Час": clock_time,
        "Похідна (Чисельно)": numeric_deriv,
        "Похідна (Аналітично)": exact_deriv,
        "Різниця": abs(numeric_deriv - exact_deriv)
    })

df_results = pd.DataFrame(results)

interpretation = {
    "10:00 (t=2)": "Похідна додатна. Аудиторія активно зростає. Потрібно готувати сервери до піку.",
    "18:00 (t=10)": "Похідна від'ємна. Активність спадає. Можна починати вивільняти ресурси.",
    "Пік": f"Максимальне навантаження о {peak_time_str}. У цей час мають працювати всі сервери."
}

print("Красивий Вивід:")
print("=" * 60)
print(f"МОМЕНТ ПІКОВОГО НАВАНТАЖЕННЯ (t*): {t_peak} годин після старту")
print(f"Астрономічний час піку: {peak_time_str}")
print("-" * 60)
print("ПОРІВНЯННЯ МЕТОДІВ ДИФЕРЕНЦІЮВАННЯ:")

formatted_df = df_results.style.format({
    "Похідна (Чисельно)": "{:.4f}",
    "Похідна (Аналітично)": "{:.4f}",
    "Різниця": "{:.2e}"
}).hide(axis="index")
display(formatted_df)
print("-" * 60)
print("БІЗНЕС-ІНТЕРПРЕТАЦІЯ:")
for key, value in interpretation.items():
    print(f"• {key}: {value}")
print("=" * 60)

print("\n\nТехнічний вивід:")
tech_data = {
    "піковий_час_t": t_peak,
    "піковий_час_формат": peak_time_str,
    "дані_порівняння": results,
    "бізнес_логіка": interpretation
}
print(tech_data)

Красивий Вивід:
МОМЕНТ ПІКОВОГО НАВАНТАЖЕННЯ (t*): 5.0 годин після старту
Астрономічний час піку: 13:00
------------------------------------------------------------
ПОРІВНЯННЯ МЕТОДІВ ДИФЕРЕНЦІЮВАННЯ:


t (год),Час,Похідна (Чисельно),Похідна (Аналітично),Різниця
2,10:00,402.1919,402.1920,1.07e-04
6,14:00,-60.2389,-60.2388,2.37e-05
10,18:00,-135.3353,-135.3353,7.22e-07


------------------------------------------------------------
БІЗНЕС-ІНТЕРПРЕТАЦІЯ:
• 10:00 (t=2): Похідна додатна. Аудиторія активно зростає. Потрібно готувати сервери до піку.
• 18:00 (t=10): Похідна від'ємна. Активність спадає. Можна починати вивільняти ресурси.
• Пік: Максимальне навантаження о 13:00. У цей час мають працювати всі сервери.


Технічний вивід:
{'піковий_час_t': 5.0, 'піковий_час_формат': '13:00', 'дані_порівняння': [{'t (год)': 2, 'Час': '10:00', 'Похідна (Чисельно)': np.float64(402.1919205007981), 'Похідна (Аналітично)': np.float64(402.19202762138355), 'Різниця': np.float64(0.00010712058542594605)}, {'t (год)': 6, 'Час': '14:00', 'Похідна (Чисельно)': np.float64(-60.238866117963006), 'Похідна (Аналітично)': np.float64(-60.23884238244046), 'Різниця': np.float64(2.3735522546530774e-05)}, {'t (год)': 10, 'Час': '18:00', 'Похідна (Чисельно)': np.float64(-135.3352839590698), 'Похідна (Аналітично)': np.float64(-135.3352832366127), 'Різниця': np.float64(7.224571163533255e-0

### Аналітичні обчислення

#### 1. Знаходження похідної
Маємо функцію:
$$f(t) = 1000 \cdot t \cdot e^{-0.2t}$$

Використовуємо правило похідної добутку $(uv)' = u'v + uv'$, де:
* $u = 1000t \Rightarrow u' = 1000$
* $v = e^{-0.2t} \Rightarrow v' = -0.2 \cdot e^{-0.2t}$ (за правилом похідної складеної функції)

Підставляємо у формулу:
$$f'(t) = 1000 \cdot e^{-0.2t} + 1000t \cdot (-0.2 \cdot e^{-0.2t})$$

Виносимо спільний множник $1000 \cdot e^{-0.2t}$ за дужки:
$$f'(t) = 1000 \cdot e^{-0.2t} \cdot (1 - 0.2t)$$

#### 2. Визначення пікового навантаження
Пік функції досягається, коли швидкість зміни дорівнює нулю (екстремум):
$$f'(t) = 0$$

$$1000 \cdot e^{-0.2t} \cdot (1 - 0.2t) = 0$$

Оскільки експонента $e^{-0.2t}$ ніколи не дорівнює нулю, прирівнюємо до нуля вираз у дужках:
$$1 - 0.2t = 0$$
$$0.2t = 1$$
$$t = \frac{1}{0.2} = 5$$

Отже, пік активності настає через **5 годин** після початку робочого дня.

**Переведення у час:**
Початок о 8:00.
$8:00 + 5 \text{ годин} = 13:00$.

#### 3. Перевірка значень (для $t=2, 6, 10$)
Підставимо значення у знайдену формулу похідної:

* **При $t=2$ (10:00):**
    $$f'(2) = 1000 \cdot e^{-0.4} \cdot (1 - 0.4) = 1000 \cdot 0.6703 \cdot 0.6 \approx 402.19$$
    *(Значення додатне $\rightarrow$ активність зростає)*

* **При $t=6$ (14:00):**
    $$f'(6) = 1000 \cdot e^{-1.2} \cdot (1 - 1.2) = 1000 \cdot 0.3012 \cdot (-0.2) \approx -60.24$$
    *(Значення від'ємне $\rightarrow$ активність спадає)*

* **При $t=10$ (18:00):**
    $$f'(10) = 1000 \cdot e^{-2.0} \cdot (1 - 2.0) = 1000 \cdot 0.1353 \cdot (-1) \approx -135.34$$
    *(Значення від'ємне $\rightarrow$ активність спадає ще швидше)*